In [1]:
import os
import pickle
import numpy as np
np.random.seed(42)
import pandas as pd
import keras
from keras.models import Model
from keras.layers import Embedding, Dense, Dropout, Input, Concatenate, Flatten, Reshape, Conv1D
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from sklearn.model_selection import train_test_split as splt
import matplotlib.pyplot as plt
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

Using TensorFlow backend.


In [2]:
dfy = pd.read_csv("MILANO_wSCORE.csv")
dfx = pd.read_csv("PLIC-milano-processed.csv", sep=";")

/home/webvalley/anaconda3/envs/score/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (14,29,36,44,51,94,104,105,112,113,120,127,130,149,158,168,237,239,240,241,248,249,256,273,298) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Thresholds are 0 - 0.01, 0.01 - 0.02, 0.02 - 0.05, 0.05 - 1
y = dfy["SCORE"].values/100
y_cat = np.zeros((y.shape[0], 4))
for i, val in enumerate(y):
    if(val < 0.01):
        y_cat[i][0] = 1
    elif(val < 0.02):
        y_cat[i][1] = 1
    elif(val < 0.05):
        y_cat[i][2] = 1
    else:
        y_cat[i][3] = 1

In [4]:
dfx = dfx.select_dtypes(exclude=['object'])
# dfx = dfx.replace(-1, np.nan).replace("-1", np.nan).replace(-1.0, np.nan)
dfx = dfx.drop(labels = ['Unnamed: 0', 'Unnamed: 0.1', 'cod pcs'], axis=1) 
dfx = dfx.dropna(how='any', axis=1)
dfx = dfx.drop([col for col in dfx.columns if "date" in col], axis = 1)

In [5]:
def joi(df, df2, sta):
    for st in sta:
        df2 = df2.join(df[st])
    return df2

dfxFloats = dfx.select_dtypes(include = 'float')
dfxInt = dfx.select_dtypes(include = 'int')
dfxFloats = joi(dfx, dfxFloats, [a for a in dfxInt.columns if (dfxInt[a].max()) > 1])
dfxInt = dfxInt.drop([a for a in dfxInt.columns if (dfxInt[a].max()) > 1], axis = 1)
dfxInt = dfxInt[dfxInt.columns[dfxInt.max()>0]]

In [6]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', -1)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')
#print_full(dfxInt.max())
#print_full(dfxFloats.max())

In [7]:
xCat = dfxInt.values + np.ones(dfxInt.shape)
xCont = dfxFloats.values

In [8]:
print(xCat.shape)
print(xCont.shape)

(5780, 79)
(5780, 93)


In [15]:
contIn = Input(shape=(xCont.shape[1],))
catIn = Input(shape=(xCat.shape[1],))

no_of_unique_cat  = len(np.unique(xCat))
embedding_size = min(np.ceil((no_of_unique_cat)/2), 50 )
embedding_size = int(embedding_size)
vocab  = no_of_unique_cat+1

catBranch = Embedding(vocab ,embedding_size)(catIn)

catBranch = Conv1D(16, 4, activation='relu')(catBranch)
catBranch = Dropout(0.4)(catBranch)
catBranch = Conv1D(4, 2, activation='relu')(catBranch)

catBranch = Flatten()(catBranch)

contBranch = Dense(32, activation='sigmoid', )(contIn)
contBranch = Dropout(0.4)(contBranch)
contBranch = Dense(16, activation='sigmoid')(contBranch)
contBranch = Dropout(0.4)(contBranch)

catBranch = Dense(16, activation='sigmoid')(catBranch)
catBranch = Dropout(0.4)(catBranch)


model = Concatenate()([contBranch, catBranch])
output = Dense(4, activation='softmax')(model)

model = Model(inputs = [contIn, catIn], outputs = [output])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
x_Cat_tr, x_Cat_ts, x_Cont_tr, x_Cont_ts, y_tr, y_ts = splt(xCat, xCont, y_cat, test_size=0.25, random_state=42)

In [ ]:
es = EarlyStopping(patience = 100, restore_best_weights = True)
history = model.fit(x=[x_Cont_tr, x_Cat_tr], y=y_tr, validation_data=([x_Cont_ts, x_Cat_ts], y_ts), epochs=500, callbacks = [es])

Train on 4335 samples, validate on 1445 samples
Epoch 1/500
4335/4335 [==============================] - 2s 395us/step - loss: 1.4514 - acc: 0.2787 - val_loss: 1.3491 - val_acc: 0.3564
Epoch 2/500
4335/4335 [==============================] - 1s 138us/step - loss: 1.3724 - acc: 0.3170 - val_loss: 1.3217 - val_acc: 0.4256
Epoch 3/500
4335/4335 [==============================] - 1s 131us/step - loss: 1.3188 - acc: 0.3739 - val_loss: 1.2426 - val_acc: 0.4498
Epoch 4/500
4335/4335 [==============================] - 1s 134us/step - loss: 1.2525 - acc: 0.4231 - val_loss: 1.1874 - val_acc: 0.4782
Epoch 5/500
4335/4335 [==============================] - 1s 136us/step - loss: 1.2129 - acc: 0.4507 - val_loss: 1.1646 - val_acc: 0.4740
Epoch 6/500
4335/4335 [==============================] - 1s 128us/step - loss: 1.1880 - acc: 0.4621 - val_loss: 1.1427 - val_acc: 0.4851
Epoch 7/500
4335/4335 [==============================] - 1s 133us/step - loss: 1.1829 - acc: 0.4694 - val_loss: 1.1310 - val_acc: 

4335/4335 [==============================] - 1s 137us/step - loss: 0.8623 - acc: 0.6256 - val_loss: 0.8096 - val_acc: 0.6505
Epoch 61/500
4335/4335 [==============================] - 1s 134us/step - loss: 0.8487 - acc: 0.6185 - val_loss: 0.7970 - val_acc: 0.6526
Epoch 62/500
4335/4335 [==============================] - 1s 134us/step - loss: 0.8369 - acc: 0.6258 - val_loss: 0.8001 - val_acc: 0.6512
Epoch 63/500
4335/4335 [==============================] - 1s 133us/step - loss: 0.8481 - acc: 0.6231 - val_loss: 0.8086 - val_acc: 0.6429
Epoch 64/500
4335/4335 [==============================] - 1s 131us/step - loss: 0.8289 - acc: 0.6332 - val_loss: 0.7918 - val_acc: 0.6484
Epoch 65/500
4335/4335 [==============================] - 1s 131us/step - loss: 0.8311 - acc: 0.6238 - val_loss: 0.7958 - val_acc: 0.6471
Epoch 66/500
4335/4335 [==============================] - 1s 134us/step - loss: 0.8409 - acc: 0.6217 - val_loss: 0.8239 - val_acc: 0.6325
Epoch 67/500
4335/4335 [=======================

4335/4335 [==============================] - 1s 131us/step - loss: 0.7395 - acc: 0.6757 - val_loss: 0.7300 - val_acc: 0.6844
Epoch 120/500
4335/4335 [==============================] - 1s 131us/step - loss: 0.7395 - acc: 0.6745 - val_loss: 0.7109 - val_acc: 0.7087
Epoch 121/500
4335/4335 [==============================] - 1s 135us/step - loss: 0.7599 - acc: 0.6708 - val_loss: 0.7364 - val_acc: 0.6720
Epoch 122/500
4335/4335 [==============================] - 1s 130us/step - loss: 0.7604 - acc: 0.6653 - val_loss: 0.7168 - val_acc: 0.6782
Epoch 123/500
4335/4335 [==============================] - 1s 124us/step - loss: 0.7496 - acc: 0.6637 - val_loss: 0.7342 - val_acc: 0.6699
Epoch 124/500
4335/4335 [==============================] - 1s 126us/step - loss: 0.7293 - acc: 0.6863 - val_loss: 0.7055 - val_acc: 0.7031
Epoch 125/500
4335/4335 [==============================] - 1s 121us/step - loss: 0.7277 - acc: 0.6807 - val_loss: 0.7040 - val_acc: 0.6927
Epoch 126/500
4335/4335 [================

4335/4335 [==============================] - 1s 129us/step - loss: 0.7182 - acc: 0.6950 - val_loss: 0.6937 - val_acc: 0.7080
Epoch 179/500
4335/4335 [==============================] - 1s 132us/step - loss: 0.7309 - acc: 0.6770 - val_loss: 0.6871 - val_acc: 0.7128
Epoch 180/500
4335/4335 [==============================] - 1s 127us/step - loss: 0.6867 - acc: 0.7054 - val_loss: 0.6643 - val_acc: 0.7273
Epoch 181/500
4335/4335 [==============================] - 1s 135us/step - loss: 0.6982 - acc: 0.7027 - val_loss: 0.6955 - val_acc: 0.6844
Epoch 182/500
4335/4335 [==============================] - 1s 132us/step - loss: 0.6871 - acc: 0.6964 - val_loss: 0.6670 - val_acc: 0.7149
Epoch 183/500
4335/4335 [==============================] - 1s 130us/step - loss: 0.7009 - acc: 0.6980 - val_loss: 0.6655 - val_acc: 0.7156
Epoch 184/500
4335/4335 [==============================] - 1s 127us/step - loss: 0.7047 - acc: 0.6939 - val_loss: 0.7001 - val_acc: 0.6747
Epoch 185/500
4335/4335 [================

4335/4335 [==============================] - 1s 131us/step - loss: 0.6484 - acc: 0.7204 - val_loss: 0.6206 - val_acc: 0.7488
Epoch 238/500
4335/4335 [==============================] - 1s 128us/step - loss: 0.6504 - acc: 0.7287 - val_loss: 0.6239 - val_acc: 0.7550
Epoch 239/500
4335/4335 [==============================] - 1s 135us/step - loss: 0.6478 - acc: 0.7232 - val_loss: 0.6250 - val_acc: 0.7522
Epoch 240/500
4335/4335 [==============================] - 1s 130us/step - loss: 0.6551 - acc: 0.7232 - val_loss: 0.6205 - val_acc: 0.7564
Epoch 241/500
4335/4335 [==============================] - 1s 137us/step - loss: 0.6682 - acc: 0.7135 - val_loss: 0.6373 - val_acc: 0.7495
Epoch 242/500
4335/4335 [==============================] - 1s 129us/step - loss: 0.6474 - acc: 0.7246 - val_loss: 0.6142 - val_acc: 0.7516
Epoch 243/500
4335/4335 [==============================] - 1s 131us/step - loss: 0.6538 - acc: 0.7170 - val_loss: 0.6271 - val_acc: 0.7543
Epoch 244/500
4335/4335 [================

4335/4335 [==============================] - 1s 130us/step - loss: 0.6223 - acc: 0.7326 - val_loss: 0.6006 - val_acc: 0.7626
Epoch 297/500
4335/4335 [==============================] - 1s 130us/step - loss: 0.6477 - acc: 0.7276 - val_loss: 0.6174 - val_acc: 0.7481
Epoch 298/500
4335/4335 [==============================] - 1s 136us/step - loss: 0.6267 - acc: 0.7347 - val_loss: 0.6321 - val_acc: 0.7329
Epoch 299/500
4335/4335 [==============================] - 1s 135us/step - loss: 0.6420 - acc: 0.7216 - val_loss: 0.6283 - val_acc: 0.7509
Epoch 300/500
4335/4335 [==============================] - 1s 130us/step - loss: 0.6356 - acc: 0.7280 - val_loss: 0.6398 - val_acc: 0.7349
Epoch 301/500
4335/4335 [==============================] - 1s 132us/step - loss: 0.6480 - acc: 0.7285 - val_loss: 0.6974 - val_acc: 0.6844
Epoch 302/500
4335/4335 [==============================] - 1s 133us/step - loss: 0.6334 - acc: 0.7396 - val_loss: 0.6067 - val_acc: 0.7543
Epoch 303/500
4335/4335 [================

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]

    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    
    print( max(history.history['val_acc']))
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)

In [14]:
#79.377